In [2]:
import numpy as np
from scipy.optimize import curve_fit
from utils import sample_observed_data_berkson, sample_observed_data_classical

In [3]:
confidence_level = 0.90
num_realizations = 100
seed = 12
folder_path = '/dcs/pg23/u1604520/mem/results/exponential_classical_unifstart/'
folder_path_simex = folder_path+'simex/exponential_classical_unifstart/'

In [4]:
scales_nu = [0.000001, 0.5,  1.0] #, 2.0]
num_scales = len(scales_nu)
c = 100
n = 200
B = 500
theta_star = np.array([1,2]) 
loc_x = 0
scale_x = 1
scale_eps = 0.5
num_methods = 3

In [5]:
def reg_func(theta,x):
    return (np.exp(theta[0] + theta[1]*x))/(1 + np.exp(theta[0] + theta[1]*x))

In [6]:
def nonlinear_model(x, a, b):
    return (np.exp(a + b*x))/(1 + np.exp(a + b*x)) #a + b*x + c*x**2

In [7]:
mses = np.zeros((num_scales, num_realizations, num_methods, len(theta_star)))
stds = np.zeros((num_scales, num_realizations, num_methods, len(theta_star)))
for j, scale_nu in enumerate(scales_nu):
    seed = 12
    counts = np.zeros((len(theta_star)))
    for r in range(num_realizations):
        seed += 1
        boot_sample = np.loadtxt(folder_path+f'sample_scale_nu{scale_nu}_c{c}_n{n}_B{B}_seed{seed}.txt')
        if scale_nu == 1.0:
            simex_sample = np.loadtxt(folder_path_simex+f'simex_scale_nu{1}_seed{seed}.txt')
        else:
            simex_sample = np.loadtxt(folder_path_simex+f'simex_scale_nu{scale_nu}_seed{seed}.txt')
        data, x = sample_observed_data_classical(reg_func, int(n), loc_x, scale_x, scale_nu, scale_eps, theta_star, seed)#np.loadtxt(folder_path+f'data_scale_nu{scale_nu}_c{c}_n{n}_B{B}_seed{seed}.txt')
        mean_boot_sample = boot_sample.mean(axis=0)
        mean_simex_sample = simex_sample.mean(axis=0)
        initial_guess = [0,0] #[0, 4]  # Initial parameter guess
        ls_estimator, _ = curve_fit(nonlinear_model, data[:,0], data[:,1], p0=initial_guess)
        mses[j, r, 0, :] = np.asarray((mean_boot_sample - theta_star)**2)
        mses[j, r, 1, :] = np.asarray((mean_simex_sample - theta_star)**2)
        mses[j, r, 2, :] = np.asarray((ls_estimator - theta_star)**2)

        # Credible interval 
        for i in range(len(theta_star)):
            count = 0
            lower_bound = np.percentile(boot_sample[:, i], (1 - confidence_level) / 2 * 100)
            upper_bound = np.percentile(boot_sample[:, i], (1 + confidence_level) / 2 * 100)
            if lower_bound <= theta_star[i] <= upper_bound:
                count += 1
            counts[i] += count
            
    # Calculate the coverage probability
    coverage_probabilities = counts / num_realizations
    mses_over_runs = np.mean(mses, axis=0)
    stds_mses_over_runs = np.std(mses, axis=0)
        
    print(f"Coverage Probability for theta_1 for ME std {scale_nu}: {coverage_probabilities[0] * 100}%")
    print(f"Coverage Probability for theta_2 for ME std {scale_nu}: {coverage_probabilities[1] * 100}%")
    #print(f"Coverage Probability for theta_3 for ME std {scale_nu}: {coverage_probabilities[2] * 100}%")
    print(f"Mean Squared error for Robust-MEM: {mses_over_runs[j, 0, :]}")
    print(f"Mean Squared error for SIMEX: {mses_over_runs[j, 1, :]}")
    print(f"Mean Squared error for Least Squares: {mses_over_runs[j, 2, :]}")
    print(f"Std - Mean Squared error for Robust-MEM: {stds_mses_over_runs[j, 0, :]}")
    print(f"Std - Mean Squared error for SIMEX: {stds_mses_over_runs[j, 1, :]}")
    print(f"Std - Mean Squared error for Least Squares: {stds_mses_over_runs[j, 2, :]}")

Coverage Probability for theta_1 for ME std 1e-06: 97.0%
Coverage Probability for theta_2 for ME std 1e-06: 100.0%
Mean Squared error for Robust-MEM: [0.01416391 0.01982347]
Mean Squared error for SIMEX: [0.02999988 0.16333361]
Mean Squared error for Least Squares: [0.00013109 0.04817631]
Std - Mean Squared error for Robust-MEM: [0.0200308  0.02803463]
Std - Mean Squared error for SIMEX: [0.04242624 0.23098861]
Std - Mean Squared error for Least Squares: [0.0001854 0.0681316]
Coverage Probability for theta_1 for ME std 0.5: 96.0%
Coverage Probability for theta_2 for ME std 0.5: 100.0%
Mean Squared error for Robust-MEM: [0.0465383  0.00910078]
Mean Squared error for SIMEX: [0.19022983 0.22831573]
Mean Squared error for Least Squares: [0.01157896 0.18757458]
Std - Mean Squared error for Robust-MEM: [0.03296121 0.01000527]
Std - Mean Squared error for SIMEX: [0.23555723 0.27031353]
Std - Mean Squared error for Least Squares: [0.00828047 0.19760123]
Coverage Probability for theta_1 for ME 

### Make a table

In [8]:
import pandas as pd

/tmp/dcs-tmp.u1604520/ipykernel_159997/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [18]:
def create_dataframe(mse_array, std_array, scale_nu, method_name):
    
    data = {'Param': ['theta_1', 'theta_2'], 'MSE': mse_array, 'Std': std_array, 'Scale_nu': scale_nu, 'Method': [method_name]*len(mse_array)}
    df = pd.DataFrame(data)
    
    return df

In [19]:
dfs = []
for i, scale_nu in enumerate(scales_nu):
    df = create_dataframe(mses_over_runs[i,0,:], stds_mses_over_runs[i,0,:], scale_nu, 'Robust-MEM (TLS)')
    dfs.append(df)
robustmem_df = pd.concat(dfs, ignore_index=True)

In [22]:
dfs = []
for i, scale_nu in enumerate(scales_nu):
    df = create_dataframe(mses_over_runs[i,1,:], stds_mses_over_runs[i,1,:], scale_nu, 'SIMEX')
    dfs.append(df)
simex_df = pd.concat(dfs, ignore_index=True)

In [23]:
dfs = []
for i, scale_nu in enumerate(scales_nu):
    df = create_dataframe(mses_over_runs[i,2,:], stds_mses_over_runs[i,2,:], scale_nu, 'LS')
    dfs.append(df)
ls_df = pd.concat(dfs, ignore_index=True)

In [24]:
df_all = pd.concat([robustmem_df, simex_df, ls_df])

In [25]:
df_all

,Param,MSE,Std,Scale_nu,Method
0,theta_1,0.029868,0.009348,0.000001,Robust-MEM (TLS)
1,theta_2,0.044356,0.011413,0.000001,Robust-MEM (TLS)
2,theta_1,0.075331,0.008035,0.500000,Robust-MEM (TLS)
3,theta_2,0.023767,0.016228,0.500000,Robust-MEM (TLS)
4,theta_1,0.037061,0.007601,1.000000,Robust-MEM (TLS)
5,theta_2,0.013547,0.005565,1.000000,Robust-MEM (TLS)
0,theta_1,0.099516,0.059184,0.000001,SIMEX
1,theta_2,0.499865,0.137549,0.000001,SIMEX
2,theta_1,0.191105,0.234853,0.500000,SIMEX
3,theta_2,0.596676,0.419813,0.500000,SIMEX
